In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import funcs
import tests
pd.set_option('display.max_columns', None)

################################################# MANUAL INPUT #######################################################
# Add at the end of the list any names that should be temporarily removed from the pairings 
## (e.g.: long-term travel, parental leave, etc.) as well as the mentees affected by the removal + their new mentors 
eids_remove_temp = []#['mickey.mouse']
mentees_impacted = []#['minnie.mouse', 'donald.duck']
new_mentors      = []#['daisy.duck', 'goofy.goof']

roster_file_path             = 'Roster Files/'
roster_generic_filename      = 'Movie Roster after %s 2023 Optimization.xlsx'

final_pairs_file_path        = 'Final Assignments/'
final_pairs_generic_filename = '%s. Coffee chat pairs - %s.xlsx'

###################################### READ INPUT & PREPARE OUTPUT FILES ##############################################
roster_original                             = funcs.read_roster_input(roster_file_path, roster_generic_filename)
roster_output_file, final_pairs_output_file = funcs.determine_output_files(roster_original, 
                                                                     roster_file_path, roster_generic_filename,
                                                                     final_pairs_file_path, final_pairs_generic_filename)

############################################### PREPROCESSING ########################################################
roster                            = funcs.temp_updates(roster_original, eids_remove_temp, mentees_impacted, new_mentors)
roster, close_contact_columns     = funcs.add_temp_columns_to_roster(roster)
roster, double_paired, odd_number = funcs.odd_number_handling(roster)

# ########################################### RUN MODEL & TESTS ########################################################
roster, people_list, person_to_index, P, B, M, R, L = funcs.prepare_optim_parameters(roster, close_contact_columns)
final_pairs_list                                    = funcs.run_optim_model(people_list, person_to_index, P, B, M, R, L)

pairs_per_person, final_pairs_df = tests.check_double_pair(final_pairs_list, double_paired)
tests.check_all_paired(people_list, pairs_per_person, odd_number)

############################################## SAVE OUTPUT ##########################################################
funcs.save_new_pairs_to_file(final_pairs_df, roster, final_pairs_output_file)
funcs.update_roster_with_new_pairs(roster_original, final_pairs_list, odd_number, roster_output_file, eids_remove_temp)